In [1]:
# It will import tensorflow and load the datasets
from config import *

In [2]:
train_set_raw.cardinality(), test_set_raw.cardinality()

(<tf.Tensor: shape=(), dtype=int64, numpy=2936>,
 <tf.Tensor: shape=(), dtype=int64, numpy=367>)

In [3]:
# tf.dataset return data in X, y format
resize_rescale = tf.keras.models.Sequential([
            tf.keras.layers.Resizing(224, 224, crop_to_aspect_ratio=True),
            tf.keras.layers.Rescaling(1./255)
])

# Data augment layer
data_augment = tf.keras.models.Sequential([
            tf.keras.layers.RandomFlip("horizontal_and_vertical"),
            tf.keras.layers.RandomRotation(0.2),
            tf.keras.layers.RandomContrast(factor=0.2)

]) 

def preprocess(df, shuffle=False, augment=False):
    tdf = df.map(lambda X,y: (resize_rescale(X), y))
    if shuffle:
        tdf = tdf.shuffle(1_000,seed=42)
    # Batch all dataset
    tdf = tdf.batch(18)
    #Augmentation on training set: training=True means it will on apply during training
    if augment:
        tdf = tdf.map(lambda X,y: (data_augment(X, training=True), y))
    return tdf.prefetch(1)

train_data = preprocess(train_set_raw,True,True)
test_data  = preprocess(test_set_raw)
valid_data = preprocess(valid_set_raw)

In [12]:
cls()
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(filters=96,kernel_size=11,strides=4,padding='valid',activation='relu',input_shape=(224,224,3)),
    tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x)),
    tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
    tf.keras.layers.Conv2D(filters=256,kernel_size=5,strides=1,padding='same',activation='relu'),
    tf.keras.layers.Lambda(lambda x: tf.nn.local_response_normalization(x)),
    tf.keras.layers.MaxPooling2D(pool_size=(3,3)),
    tf.keras.layers.Conv2D(filters=384,kernel_size=3,strides=1,padding='same',activation='relu'),
    tf.keras.layers.Conv2D(filters=384,kernel_size=3,strides=1,padding='same',activation='relu'),
    tf.keras.layers.Conv2D(filters=256,kernel_size=3,strides=1,padding='same',activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(4096,activation='relu'),
    tf.keras.layers.Dropout(.5),
    tf.keras.layers.Dense(4096,activation='relu'),
    tf.keras.layers.Dropout(.5),
    tf.keras.layers.Dense(5,activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 54, 54, 96)        34944     
_________________________________________________________________
lambda (Lambda)              (None, 54, 54, 96)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 18, 18, 96)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 18, 18, 256)       614656    
_________________________________________________________________
lambda_1 (Lambda)            (None, 18, 18, 256)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 256)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 6, 6, 384)         8

In [11]:
model.compile(loss='sparse_categorical_crossentropy',optimizer='nadam',metrics=['accuracy'])
hist = model.fit(train_data,validation_data=(valid_data), epochs=1)
score = model.evaluate(test_data)
score

21/21 [==============================] - 3s 138ms/step - loss: 1.6017 - accuracy: 0.2425


[1.6017495393753052, 0.24250681698322296]